In [20]:
import pandas as pd
import numpy as np

# Retrieve River Flows
flow = pd.read_csv('C:/Users/Scott/Desktop/Projects/River_Data/Ingestion/CurrentWater.csv')
# Retrieve Precip
rain = pd.read_csv('C:/Users/Scott/Desktop/Projects/River_Data/Ingestion/CurrentPrecip.csv',index_col=0)
# Retrieve Forecast
rainforecast = pd.read_csv('C:/Users/Scott/Desktop/Projects/River_Data/Ingestion/WeatherForecast.csv',index_col=0)
# Retrieve River References
reference = pd.read_csv('C:/Users/Scott/Desktop/Projects/River_Data/RiverReferenceTable.csv')


# Join on Date
flow = flow.set_index('Name').join(reference[['USGS Name','Name']].set_index('USGS Name')).reset_index()
flow['CFS'] = flow.Value
flow['DateTime'] = pd.to_datetime(flow.DateTime)
flow['Date'] = flow.DateTime.dt.date
flow = flow[['Date','Name','CFS']].groupby(['Date','Name']).mean().reset_index()
flow['Date_Name'] = flow.Date.astype(str)+'_'+flow.Name


rain = rain.drop_duplicates()
rain['Date_Name'] = rain.Date+'_'+rain.Name
rain['Date'] = pd.to_datetime(rain.Date)
rain['Date'] = rain.Date.dt.date
rain = rain[['Date_Name','Date','Name','Precip']].groupby(['Date','Name','Date_Name']).mean().reset_index()

df = flow.set_index('Date_Name').join(rain[['Date_Name','Precip']].set_index('Date_Name'))
RiverList = df.Name.unique()
full = df.loc[df.Precip.isnull()==False]

# Lag Precip and Forecast data by correlation strength to river flow
RiverData = pd.DataFrame()
for n in RiverList:
    s = 0
    CorrList = list()
    while s < 10:
        d = full[['CFS','Precip']].loc[full.Name==n]
        d['Precip'] = d['Precip'].shift(s)
        r = d.dropna()
        corr = r.Precip.corr(r.CFS)
        CorrList.append([s,corr])
        s+=1
    Corrs = pd.DataFrame(CorrList,columns=['PrecipLag','Corr'])
    BestShift = Corrs.sort_values(by='Corr',ascending=False).head(1)
    BestShift['Name'] = n
    RiverData = RiverData.append(BestShift)

    
# Format data like example below
headers = ['Date','RiverName','DayQuality','DayAvgCFS','Precip','NextDayForecast','Forecast_2','Forecast_3','Forecast_4','Forecast_5','Forecast_6','Forecast_7','Forecast_8','Forecast_9','Forecast_10']
d = [['11-01-2022',0,0.88,500,2,0,0,2,3,4,5,0,2,0,3]]
model = pd.DataFrame(data=d,columns=headers)


# Bring in river level data
rivers = pd.read_csv('C:/Users/Scott/Desktop/Projects/River_Data/RiverReferenceTable.csv')[['Name','RiverName']]

# Create correlation coefficients for the following attributes
# corr(CFS & Precip)
# corr(CFS & Forecast)
# corr(Precip & Forecast)

# Append those coefficients to river level dataset
rivers['River__Rain_Coeff'] = None
rivers['River__Forecast_Coeff'] = None
rivers['Rain__Forecast_Coeff'] = None
rivers.head()

c:\Users\Scott\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2683: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
c:\Users\Scott\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


,Name,RiverName,River__Rain_Coeff,River__Forecast_Coeff,Rain__Forecast_Coeff
0,Russian River HaciendaBridge,Russian,None,None,None
1,Navarro River,Navarro,None,None,None
2,Noyo River,Noyo,None,None,None
3,San Lorenzo River Upper,San Lorenzo,None,None,None
4,Mad River,Mad,None,None,None


In [21]:
full.head()

,Date,Name,CFS,Precip
Date_Name,,,,
2021-12-06_East Fork Carson,2021-12-06,East Fork Carson,85.530208,3.80
2021-12-06_East Walker,2021-12-06,East Walker,23.458333,0.00
2021-12-06_Eel River Fort Seward,2021-12-06,Eel River Fort Seward,533.489583,0.00
2021-12-06_Eel River Scotia,2021-12-06,Eel River Scotia,880.395833,2.80
2021-12-06_Little Truckee River,2021-12-06,Little Truckee River,34.364583,1.25


In [22]:
RiverData.head()

,PrecipLag,Corr,Name
7,7,0.770553,East Fork Carson
8,8,0.448832,East Walker
1,1,0.555460,Eel River Fort Seward
2,2,0.539900,Eel River Scotia
0,0,NaN,Hat Creek


In [23]:
rainforecast['RainCode'] = rainforecast.LightRain.astype(str)+rainforecast.Rain.astype(str)+rainforecast.HeavyRain.astype(str)
def forecast_encoder(x):
    if x == '000': return 0
    elif x[0] == '1': return 1
    elif x[1] == '1': return 2
    elif x[2] == '1': return 3
rainforecast['forecast_code'] = rainforecast.RainCode.apply(lambda x: forecast_encoder(x))


In [24]:
rainforecast.StationName.unique()

array(['East Fork Carson', 'East Walker', nan, 'Eel River Fort Seward',
       'Eel River Scotia', 'Hat Creek', 'Little Truckee River',
       'Mad River', 'Mattole River Ettersburg', 'Mattole River Petrolia',
       'Yuba River', 'Navarro River', 'Nestucca River Beaver',
       'North Fork American River', 'North Umpqua River Winchester',
       'North Umpquar River Idlelyd', 'Noyo River', 'Pit River',
       'Putah Creek', 'Redwood Creek Orick', 'Rogue River Grants Pass',
       'Rogue River Agness', 'Russian River HaciendaBridge',
       'San Lorenzo River Upper', 'Smith River CC',
       'Trinity River Junction City', 'Trinity River Lewiston',
       'Trinity River Burnt Ranch', 'Truckee River Farad',
       'Umpqua River Elkton', 'Van Duzen River Bridgeville',
       'Yuba River Marysville', 'San Lorenzo River'], dtype=object)

In [30]:
rainforecast.loc[rainforecast.StationName=='Rogue River Agness'].tail(20)

,Station,name,startTime,endTime,temperature,windDirection,WindMph,LightRain,Rain,HeavyRain,StationName,Date_Name,RainCode,forecast_code
2318,"ROGUE RIVER NEAR AGNESS, OR",Friday Night,2022-04-02 01:00:00+00:00,2022-04-02 13:00:00+00:00,39,NNE,10,0,0,0,Rogue River Agness,2022-04-02_Rogue River Agness,000,0
2319,"ROGUE RIVER NEAR AGNESS, OR",Saturday,2022-04-02 13:00:00+00:00,2022-04-03 01:00:00+00:00,62,NNE,9,0,0,0,Rogue River Agness,2022-04-02_Rogue River Agness,000,0
2320,"ROGUE RIVER NEAR AGNESS, OR",Saturday Night,2022-04-03 01:00:00+00:00,2022-04-03 13:00:00+00:00,39,NNE,9,1,0,0,Rogue River Agness,2022-04-03_Rogue River Agness,100,1
2321,"ROGUE RIVER NEAR AGNESS, OR",Sunday,2022-04-03 13:00:00+00:00,2022-04-04 01:00:00+00:00,55,SW,12,1,0,0,Rogue River Agness,2022-04-03_Rogue River Agness,100,1
2322,"ROGUE RIVER NEAR AGNESS, OR",Sunday Night,2022-04-04 01:00:00+00:00,2022-04-04 13:00:00+00:00,40,SW,12,1,0,0,Rogue River Agness,2022-04-04_Rogue River Agness,100,1
2323,"ROGUE RIVER NEAR AGNESS, OR",Monday,2022-04-04 13:00:00+00:00,2022-04-05 01:00:00+00:00,54,SW,10,1,0,0,Rogue River Agness,2022-04-04_Rogue River Agness,100,1
280,"ROGUE RIVER NEAR AGNESS, OR",Tonight,2022-12-02 02:00:00+00:00,2022-12-02 14:00:00+00:00,29,S,7,0,0,0,Rogue River Agness,2022-12-02_Rogue River Agness,000,0
281,"ROGUE RIVER NEAR AGNESS, OR",Friday,2022-12-02 14:00:00+00:00,2022-12-03 02:00:00+00:00,41,S,14,0,0,0,Rogue River Agness,2022-12-02_Rogue River Agness,000,0
282,"ROGUE RIVER NEAR AGNESS, OR",Friday Night,2022-12-03 02:00:00+00:00,2022-12-03 14:00:00+00:00,32,SSE,14,0,0,0,Rogue River Agness,2022-12-03_Rogue River Agness,000,0
283,"ROGUE RIVER NEAR AGNESS, OR",Saturday,2022-12-03 14:00:00+00:00,2022-12-04 02:00:00+00:00,45,SE,9,0,0,0,Rogue River Agness,2022-12-03_Rogue River Agness,000,0
